In [ ]:
from stmpy import Driver, Machine
from threading import Thread
import paho.mqtt.client as mqtt
import json
from stmpy import Machine, Driver
import ipywidgets as widgets
from IPython.display import display
import random



class Server:
    def __init__(self):
        self.button_ask = widgets.Button(description="Ask")
        self.button_ask.on_click(self.on_button_ask)
        self.answered = ""
        self.message = ""
        # text field
        self.text = widgets.Text(value='', placeholder='', description='String:', disabled=False)
        display(self.text, self.button_ask)
    
    def on_button_ask(self, b):
        self.stm.send('ask') # <---- here we send a message
        print(self.text.value)
        
    def on_idle(self):
        print("idle")
        self.text.set_trait(name='value', value='idle...')

    def on_asking(self):
        print("asking")
        self.text.set_trait(name='value', value='asking!')
        
    def on_received(self, message):
        print("received")
        self.answered = self.message
        self.text.set_trait(name='value', value=f"{self.message}")
    
    def send_mqtt_tick(self):
        print("Tick! {}".format(self.ticks))
        self.ticks = self.ticks + 1
        self.mqtt_client.publish("tick", self.ticks)

    def send_mqtt_tock(self):
        print("Tock! {}".format(self.tocks))
        self.tocks = self.tocks + 1
        self.mqtt_client.publish("tock", self.ticks)

# initial transition
t0 = {'source': 'initial',
      'target': 'idle'}

# transitions
t1 = {'trigger':'ask', 
      'source':'idle', 
      'target':'asking'}
t2 = {'trigger':'t', 
      'source':'asking', 
      'target':'idle'}
t3 = {'trigger':'t', 
      'source':'received', 
      'target':'idle'}
t4 = {'trigger':'message', 
      'source':'asking', 
      'target':'received'}
t5 = {'trigger':'reset', 
      'source':'asking', 
      'target':'idle'}

 
# the states:
idle = {'name': 'idle',
       'entry': 'on_idle'}

asking = {'name': 'asking',
        'entry': 'on_asking; start_timer("t", 20000)'}

received = {'name': 'received',
        'entry': 'on_received; stop_timer("t")'}



In [3]:
server = Server()

Text(value='', description='String:', placeholder='')

Button(description='Ask', style=ButtonStyle())